In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [1]:
import pandas

In [2]:
#path = ''
df_cons = pandas.read_parquet(f"consolidado_2020_a_2024.parquet")

In [3]:
# Remover a coluna 'tipo_cliente'
df_cons = df_cons.drop(columns=['tipo_cliente'])


In [4]:
df_cons["numero_de_operacoes"] = (
    df_cons["numero_de_operacoes"]
    .str.replace(r"[^\d]", "", regex=True)  # Remove tudo que não for número
    .astype(int)  # Converte para inteiro
)

In [5]:
df_cons

,data_base,uf,cliente,ocupacao,cnae_secao,porte,modalidade,numero_de_operacoes,a_vencer_ate_90_dias,carteira_ativa,carteira_inadimplida_arrastada,ativo_problematico
0,2020-01-31,AC,PF,PF - Aposentado/pensionista,-,PF - Acima de 20 salários mínimos,PF - Cartão de crédito,974,4758621.08,6186180.64,91888.44,157442.87
1,2020-01-31,AC,PF,PF - Aposentado/pensionista,-,PF - Acima de 20 salários mínimos,PF - Cartão de crédito,15,74.78,770.72,0.00,770.72
2,2020-01-31,AC,PF,PF - Aposentado/pensionista,-,PF - Acima de 20 salários mínimos,PF - Empréstimo com consignação em folha,797,1915326.67,32180285.36,171468.42,171468.42
3,2020-01-31,AC,PF,PF - Aposentado/pensionista,-,PF - Acima de 20 salários mínimos,PF - Empréstimo com consignação em folha,15,0.00,75981.96,75981.96,75981.96
4,2020-01-31,AC,PF,PF - Aposentado/pensionista,-,PF - Acima de 20 salários mínimos,PF - Empréstimo sem consignação em folha,198,351985.99,3153279.41,45870.20,118921.38
...,...,...,...,...,...,...,...,...,...,...,...,...
50369260,2024-12-31,TO,PJ,-,"PJ - Transporte, armazenagem e correio",PJ - Médio,PJ - Outros créditos,15,17264.64,17264.64,0.00,0.00
50369261,2024-12-31,TO,PJ,-,"PJ - Água, esgoto, atividades de gestão de res...",PJ - Indisponível,PJ - Outros créditos,15,1425.85,1425.85,0.00,0.00
50369262,2024-12-31,TO,PJ,-,"PJ - Água, esgoto, atividades de gestão de res...",PJ - Micro,PJ - Outros créditos,15,0.00,48081.37,48081.37,48081.37
50369263,2024-12-31,TO,PJ,-,"PJ - Água, esgoto, atividades de gestão de res...",PJ - Médio,PJ - Outros créditos,15,1141.88,1141.88,0.00,0.00


In [9]:
df_cons_agg = df_cons.groupby(["data_base", "uf", "cliente", "ocupacao", "cnae_secao", "porte", "modalidade"]).agg({
     "a_vencer_ate_90_dias": "sum",
     "numero_de_operacoes": "sum",
     "carteira_ativa":  "sum",
     "carteira_inadimplida_arrastada":  "sum",
     "ativo_problematico": "sum"
}).reset_index()

In [11]:
df_cons_agg

,data_base,uf,cliente,ocupacao,cnae_secao,porte,modalidade,a_vencer_ate_90_dias,numero_de_operacoes,carteira_ativa,carteira_inadimplida_arrastada,ativo_problematico
0,2020-01-31,AC,PF,PF - Aposentado/pensionista,-,PF - Acima de 20 salários mínimos,PF - Cartão de crédito,5292551.34,1221,7134029.36,110554.13,200628.59
1,2020-01-31,AC,PF,PF - Aposentado/pensionista,-,PF - Acima de 20 salários mínimos,PF - Empréstimo com consignação em folha,2560250.69,1463,40406261.43,479130.32,479130.32
2,2020-01-31,AC,PF,PF - Aposentado/pensionista,-,PF - Acima de 20 salários mínimos,PF - Empréstimo sem consignação em folha,487161.78,289,5202472.57,95945.45,199366.41
3,2020-01-31,AC,PF,PF - Aposentado/pensionista,-,PF - Acima de 20 salários mínimos,PF - Habitacional,216562.86,34,5688298.55,0.00,174443.77
4,2020-01-31,AC,PF,PF - Aposentado/pensionista,-,PF - Acima de 20 salários mínimos,PF - Outros créditos,330918.45,261,1941302.78,38105.42,308276.42
...,...,...,...,...,...,...,...,...,...,...,...,...
1614739,2024-12-31,TO,PJ,-,"PJ - Água, esgoto, atividades de gestão de res...",PJ - Pequeno,PJ - Cheque especial e conta garantida,259708.92,75,324275.62,0.00,59615.32
1614740,2024-12-31,TO,PJ,-,"PJ - Água, esgoto, atividades de gestão de res...",PJ - Pequeno,PJ - Financiamento de infraestrutura/desenvolv...,69968.15,30,433905.12,0.00,0.00
1614741,2024-12-31,TO,PJ,-,"PJ - Água, esgoto, atividades de gestão de res...",PJ - Pequeno,PJ - Investimento,722649.97,120,5747517.09,206370.07,249310.16
1614742,2024-12-31,TO,PJ,-,"PJ - Água, esgoto, atividades de gestão de res...",PJ - Pequeno,PJ - Operações com recebíveis,6116.63,30,11699.50,0.00,0.00


In [12]:
df_cons_agg.count()

data_base                         1614744
uf                                1614744
cliente                           1614744
ocupacao                          1614744
cnae_secao                        1614744
porte                             1614744
modalidade                        1614744
a_vencer_ate_90_dias              1614744
numero_de_operacoes               1614744
carteira_ativa                    1614744
carteira_inadimplida_arrastada    1614744
ativo_problematico                1614744
dtype: int64

In [16]:
df_cons.count()

data_base                         49462493
uf                                50369265
cliente                           50369265
ocupacao                          50369265
cnae_secao                        50369265
porte                             50369265
modalidade                        50369265
numero_de_operacoes               50369265
a_vencer_ate_90_dias              50369265
carteira_ativa                    50369265
carteira_inadimplida_arrastada    50369265
ativo_problematico                50369265
tipo_cliente                      50369265
dtype: int64

In [13]:
# Salvar o DataFrame em um arquivo CSV
df_cons_agg.to_csv('df_cons_agg.csv', index=False)  # index=False para não salvar o índice

print("DataFrame salvo em 'df_cons_agg.csv'")

DataFrame salvo em 'df_cons_agg.csv'


In [ ]:
# Defina o número de registros por arquivo
registros_por_arquivo = 1000000

# Calcule o número total de arquivos necessários
num_arquivos = len(df) // registros_por_arquivo + (1 if len(df) % registros_por_arquivo > 0 else 0)

# Salvar o DataFrame em arquivos CSV particionados
for i in range(num_arquivos):
    # Calcule o índice inicial e final para a partição
    inicio = i * registros_por_arquivo
    fim = inicio + registros_por_arquivo
    
    # Crie um DataFrame para a partição
    df_particao = df.iloc[inicio:fim]
    
    # Salve a partição em um arquivo CSV
    df_particao.to_csv(f'parte_{i + 1}.csv', index=False)